# cleaning

In [4]:
import requests 
import json
import os
from getpass import getpass
import pandas as pd
from pandas import json_normalize
#import tweepy
import time
from datetime import datetime
import sys
sys.path.append('../')
import os
from src import functions as fn 
from riotwatcher import LolWatcher, ApiError

In [ ]:
df = pd.read_csv("../data/player_data.csv")

In [10]:
pd.set_option("display.max_columns", 108)
df.sample(4)

,gameDuration,gameMode,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
8,1609061,ARAM,27,0,1,27901,18,53,Blitzcrank,0,2,2848,2848,2848,38044,11,0,1,0,True,False,False,False,False,False,15601,13350,Invalid,0,1,0,6656,3157,3001,3111,4629,3801,2052,18,3,9,MIDDLE,0,2,2,241,37213,21216,15502,0,0,0,1,0,0,1,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",9452,4491,15922,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SOLO,0,57,44,46,39,33,32,7,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,405,jalex141,False,100,NaN,71,1600,47836,26693,0,34554,3393,0,14,222,312,1,0,1170,985,3130,2,3,2,0,0,0,0,0,True
2,1125748,ARAM,14,0,0,18332,17,53,Blitzcrank,0,2,0,0,0,18471,12,0,1,0,True,False,False,False,False,True,13011,12100,Invalid,0,0,1,6655,3157,3020,3102,1058,1058,2052,16,3,10,NONE,0,2,2,134,30188,16192,9091,0,0,0,0,0,0,1,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",3296,1473,13122,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,34,21,23,18,5,4,26,32,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,405,jalex141,False,100,NaN,31,1117,37015,18515,0,22780,1740,0,22,80,284,1,0,3530,850,566,0,0,4,0,0,0,0,0,False
10,1598082,CLASSIC,11,0,1,11127,13,236,Lucian,0,2,3112,17455,3112,3725,1,0,2,1,False,False,False,False,False,True,12557,9950,BOTTOM,2,2,0,6672,0,3006,6676,1053,1043,3340,16,1,8,BOTTOM,492,7,3,1180,4438,1780,3138,4,0,0,0,0,0,9,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",97885,10934,6150,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,CARRY,0,36,28,28,9,3,7,2,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,405,jalex141,False,200,BOTTOM,1,1589,115366,13606,0,9563,2535,555,169,15,36,3,1,13042,891,274,1,2,1,0,18,0,4,8,True
19,1515366,ARAM,18,0,3,27487,18,10,Kayle,0,2,5812,5812,5812,25728,8,0,1,0,False,False,False,False,False,False,18612,16000,Invalid,1,1,1,6673,3006,3091,3094,3153,3085,2052,21,3,9,MIDDLE,500,3,2,215,71163,12540,18362,0,0,0,1,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",108095,15747,13787,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,54,63,40,8,6,21,5,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,404,jalex141,False,200,NaN,3,1507,180804,28287,0,34034,7346,2249,162,52,206,5,0,1545,0,1883,4,4,2,0,0,0,0,0,True
